# Exemplo 6: Finanças - Análise do Mercado de Ações em Tempo Real

Este notebook demonstra o cálculo de médias móveis (SMA) em tempo real para ações usando **Spark Streaming**.

**Cenário**: Calcular a média móvel de 10 segundos para preços de ações (AAPL, GOOGL, MSFT) para identificar tendências.

## 1. Configuração

In [ ]:
# Instalar Java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# Baixar e Instalar Spark
!wget https://archive.apache.org/dist/spark/spark-3.5.0/spark-3.5.0-bin-hadoop3.tgz && tar xf spark-3.5.0-bin-hadoop3.tgz

# Baixar e Instalar Kafka
!wget https://archive.apache.org/dist/kafka/3.6.1/kafka_2.13-3.6.1.tgz && tar xf kafka_2.13-3.6.1.tgz

# Instalar pacotes Python
!pip install -q findspark pyspark kafka-python

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.0-bin-hadoop3"
import findspark
findspark.init()

## 2. Iniciar Kafka

In [ ]:
%%bash
cd kafka_2.13-3.6.1
bin/zookeeper-server-start.sh -daemon config/zookeeper.properties
sleep 5
bin/kafka-server-start.sh -daemon config/server.properties
sleep 5
bin/kafka-topics.sh --create --topic stock-prices --bootstrap-server localhost:9092 --replication-factor 1 --partitions 1

## 3. Simulador de Mercado

In [ ]:
import json
import time
import random
from kafka import KafkaProducer
import threading
from datetime import datetime

def generate_stock_data():
    producer = KafkaProducer(bootstrap_servers=['localhost:9092'],
                             value_serializer=lambda x: json.dumps(x).encode('utf-8'))
    stocks = ['AAPL', 'GOOGL', 'MSFT', 'AMZN']
    
    try:
        for _ in range(200):
            for stock in stocks:
                price = random.uniform(100, 200)
                data = {
                    'symbol': stock,
                    'price': price,
                    'timestamp': datetime.now().isoformat()
                }
                producer.send('stock-prices', value=data)
            time.sleep(0.5)
    finally:
        producer.close()

thread = threading.Thread(target=generate_stock_data)
thread.start()

## 4. Cálculo de Médias Móveis (Processamento)

In [ ]:
%%writefile kafka_consumer.py
from pyspark.sql import SparkSession
from pyspark.sql.functions import from_json, col, window, avg
from pyspark.sql.types import StructType, StructField, StringType, FloatType, TimestampType

spark = SparkSession.builder.appName("StockAnalysis").getOrCreate()
spark.conf.set("spark.sql.shuffle.partitions", "2") # Otimizar para local

schema = StructType([
    StructField("symbol", StringType()),
    StructField("price", FloatType()),
    StructField("timestamp", TimestampType())
])

df = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("subscribe", "stock-prices") \
    .load()

stocks = df.select(from_json(col("value").cast("string"), schema).alias("data")).select("data.*")

# Janela deslizante de 10 segundos, atualizada a cada 5 segundos
windowed_avg = stocks \
    .groupBy(
        window(col("timestamp"), "10 seconds", "5 seconds"),
        col("symbol")
    ) \
    .agg(avg("price").alias("average_price"))

query = windowed_avg.writeStream \
    .outputMode("complete") \
    .format("console") \
    .option("truncate", "false") \
    .start()


query.awaitTermination()

In [ ]:
!spark-submit --packages org.apache.spark:spark-sql-kafka-0-10_2.12:3.2.1 kafka_consumer.py